# **SYDE 552 Notebook**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd 'gdrive'/'MyDrive'/'4th Year'/'SYDE 552'/'SYDE 552 Project'

Mounted at /content/gdrive
/content/gdrive/MyDrive/4th Year/SYDE 552/SYDE 552 Project


In [ ]:
!apt-get install -qq ffmpeg freeglut3-dev xvfb  # For visualization

!pip install -q git+https://github.com/DLR-RM/stable-baselines3#egg=stable-baselines3[extra]

In [ ]:
import gym
import numpy as np
import torch as th
import matplotlib.pyplot as plt

from stable_baselines3 import DQN, A2C
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
env = gym.make("CartPole-v1")
# env = gym.make("LunarLanderContinuous-v2")
# env = gym.make("BreakoutNoFrameskip-v4")

/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


In [ ]:
tensorboard_log = "data_original/tb/"

In [ ]:
model = DQN("MlpPolicy",
            env,
            verbose=1,
            train_freq=16,
            gradient_steps=8,
            gamma=0.99,
            exploration_fraction=0.2,
            exploration_final_eps=0.07,
            target_update_interval=600,
            learning_starts=1000,
            buffer_size=10000,
            batch_size=128,
            learning_rate=4e-3,
            policy_kwargs=dict(net_arch=[256, 256]),
            tensorboard_log=tensorboard_log,
            seed=2)

# model = A2C("MlpPolicy",
#             env,
#             verbose=0,
#             gamma=0.99,
#             learning_rate=4e-3,
#             policy_kwargs=dict(net_arch=[256, 256]),
#             tensorboard_log=tensorboard_log,
#             seed=2)

# model = PPO("MlpPolicy",
#             env,
#             verbose=0,
#             gamma=0.99,
#             learning_rate=4e-3,
#             policy_kwargs=dict(net_arch=[256, 256]),
#             tensorboard_log=tensorboard_log,
#             seed=2)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
mean_reward, std_reward = evaluate_policy(model, model.get_env(), deterministic=True, n_eval_episodes=20)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
# Optional: Monitor training in tensorboard
%load_ext tensorboard
%tensorboard --logdir $tensorboard_log

In [ ]:
model.learn(int(1.2e5), log_interval=10)

In [ ]:
mean_reward, std_reward = evaluate_policy(model, model.get_env(), deterministic=True, n_eval_episodes=4)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'
import base64
from pathlib import Path
from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  eval_env = DummyVecEnv([lambda: env_id])
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs, deterministic=False)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [ ]:
env2 = gym.make("CartPole-v1")
env = CartPoleEnv()
record_video(CartPoleEnv(), model, video_length=500, prefix='dqn-cartpole')
show_videos('videos', prefix='dqn-cartpole')